In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
import pandas as pd
import re
import json
import h2o


from functools import lru_cache

from utils import get_data, Config

from tree import make_tree

In [4]:
def clean_url(url):
    return re.sub(r"browser_id=(\d+)", "browser_id=2", url)

def make_clickable(url):
    # Clickable links for debugging; set browser_id=2 (unknown) to not count them in our real data collection
    url = clean_url(url)
    return f'<a href="{url}" target="_blank">{url}</a>'

In [5]:
@lru_cache(maxsize=None)
def id_to_browser(id):
    return " ".join(df.loc[df["browser_id"] == id].iloc[0][["name", "os", "version", "automation_mode", "headless_mode"]].to_list())

## Load Data + Preprocessing

In [6]:
# Load all data
initial_data = """
SELECT "Result".*, 
"Response".raw_header, "Response".status_code, "Response".label, "Response".resp_type,
"Browser".name, "Browser".version, "Browser".headless_mode, "Browser".os, "Browser".automation_mode, "Browser".add_info
FROM "Result"
JOIN "Response" ON "Result".response_id = "Response".id JOIN "Browser" ON "Result".browser_id = "Browser".id
WHERE "Response".created_at <= DATE('2023-11-22') and "Browser".name != 'Unknown';
"""
df = get_data(Config(), initial_data)

Connecting to the PostgreSQL database...
Connection successful


In [7]:
# Extra columns
df["outcome_str"] = df["outcome_value"].fillna("None").astype(str)

# Unify outcomes that are the same
# Firefox: {'error': 'object "TypeError: NetworkError when attempting to fetch resource."', 'headers': ''} Chromium: {'error': 'object "TypeError: Failed to fetch"', 'headers': ''}
# Safari: {'error': 'object "TypeError: Load failed"', 'headers': ''}
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: NetworkError when attempting to fetch resource.\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")
df["outcome_str"] = df["outcome_str"].replace("{'error': 'object \"TypeError: Load failed\"', 'headers': ''}", "{'error': 'object \"TypeError: Failed to fetch\"', 'headers': ''}")

# For document referrer we do not want to know the exact resp_id and count
# Maybe we even only want to know whether it is a origin or the full URl?
df['outcome_str'] = df['outcome_str'].replace(r'resp_id=\d+', 'resp_id=<resp_id>', regex=True)
df['outcome_str'] = df['outcome_str'].replace(r'count=\d+', 'count=<count>', regex=True)

df["clean_url"] = df["full_url"].apply(clean_url)

df["browser"] = df["browser_id"].apply(id_to_browser)
df["org_origin"] = df["org_scheme"] + "://" + df["org_host"]
df["resp_origin"] = df["resp_scheme"] + "://" + df["resp_host"]

In [8]:
df["outcome_str"].unique()

array(["{'error': 'null', 'headers': 'access-control-allow-orgin,content-length,date,server,test,'}",
       '{\'error\': \'object "TypeError: Failed to fetch"\', \'headers\': \'\'}',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-orgin,content-length,date,server,test,'}",
       'message send',
       "{'error': 'null', 'headers': 'access-control-allow-credentials,access-control-allow-headers,access-control-allow-methods,access-control-allow-orgin,access-control-expose-headers,content-length,date,server,test,'}",
       "{'error': 'null', 'headers': 'cache-control,content-type,'}",
       'message timeout', 'fullscreenEnabled: true',
       'fullscreenEnabled: false', 'window.originAgentCluster: false',
       'window.originAgentCluster: true', "{'requestStart != 0': True}",
       "{'requestStart != 0': False}",
       'document.refe

## Overview

In [9]:
display(df.head(2))
display(df.describe())
display(df.columns)

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
0,1,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_GET,0,None,None,http,sub.headers.websec.saarland,http,...,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,chrome Ubuntu 22.04 119 selenium headless-new,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland
1,2,<class 'dict'>,"{'error': 'null', 'headers': 'access-control-a...",fetch_TEST,0,None,None,http,sub.headers.websec.saarland,http,...,119,headless-new,Ubuntu 22.04,selenium,,"{'error': 'null', 'headers': 'access-control-a...",http://sub.headers.websec.saarland/_hp/tests/f...,chrome Ubuntu 22.04 119 selenium headless-new,http://sub.headers.websec.saarland,http://sub.headers.websec.saarland


,id,test_status,browser_id,testcase_id,response_id,created_at,updated_at,status_code
count,303624.000000,303624.000000,303624.000000,303624.0,303624.000000,303624,303624,303624.000000
mean,151964.337272,0.000593,54.184834,1.0,78.185190,2023-11-21 14:48:46.707897856,2023-11-21 14:48:46.707899392,216.031144
min,1.000000,0.000000,5.000000,1.0,3.000000,2023-11-21 10:34:47.097638,2023-11-21 10:34:47.097641,200.000000
25%,75906.750000,0.000000,14.000000,1.0,23.000000,2023-11-21 12:13:48.703102720,2023-11-21 12:13:48.703103744,200.000000
50%,151812.500000,0.000000,60.000000,1.0,47.000000,2023-11-21 13:52:23.833979392,2023-11-21 13:52:23.833979392,200.000000
75%,228142.250000,0.000000,74.000000,1.0,151.000000,2023-11-21 17:32:09.489758464,2023-11-21 17:32:09.489761024,200.000000
max,308044.000000,2.000000,116.000000,1.0,217.000000,2023-11-22 14:01:43.683542,2023-11-22 14:01:43.683544,302.000000
std,87824.253431,0.034429,35.891441,0.0,66.615821,NaN,NaN,37.123896


Index(['id', 'outcome_type', 'outcome_value', 'test_name', 'test_status',
       'test_message', 'test_stack', 'org_scheme', 'org_host', 'resp_scheme',
       'resp_host', 'relation_info', 'browser_id', 'testcase_id',
       'response_id', 'status', 'full_url', 'created_at', 'updated_at',
       'raw_header', 'status_code', 'label', 'resp_type', 'name', 'version',
       'headless_mode', 'os', 'automation_mode', 'add_info', 'outcome_str',
       'clean_url', 'browser', 'org_origin', 'resp_origin'],
      dtype='object')

In [10]:
# Collected results per browser
df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["id"].count().sort_values(ascending=False).to_frame()

id
name    version                   os           headless_mode automation_mode       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium         20912
                                               xvfb          selenium         20912
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium         20912
                                               xvfb          selenium         20912
chrome  119                       Ubuntu 22.04 headless-new  selenium         20912
edge    119                       Ubuntu 22.04 headless-new  selenium         20912
chrome  119                       Ubuntu 22.04 xvfb          selenium         20912
firefox 119                       Ubuntu 22.04 headless      selenium         20912
                                               xvfb          selenium         20912
edge    119                       Ubuntu 22.04 xvfb          selenium         20912
safari  17.0                      macOS 14.0   real          selenium         10840
edge    119                       macOS 14.0   headless-new  selenium         10472
chrome  119                       macOS 14.0   headless-new  selenium         10456
brave   1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium         10456
                                               real          selenium         10456
chrome  119                       macOS 14.0   real          selenium         10456
edge    119                       macOS 14.0   real          selenium         10456
firefox 119                       macOS 14.0   headless      selenium         10456
                                               real          selenium         10456

In [11]:
# Timeouts
display(df.groupby(["name", "version", "os", "headless_mode", "automation_mode"])["test_status"].value_counts().to_frame())

display(df.loc[df["test_status"] == 2].head(3))

count
name    version                   os           headless_mode automation_mode test_status       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        0            20912
                                               xvfb          selenium        0            20912
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        0            20912
                                               xvfb          selenium        0            20912
        1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium        0            10456
                                               real          selenium        0            10456
chrome  119                       Ubuntu 22.04 headless-new  selenium        0            20912
                                               xvfb          selenium        0            20912
                                  macOS 14.0   headless-new  selenium        0            10456
                                               real          selenium        0            10456
edge    119                       Ubuntu 22.04 headless-new  selenium        0            20912
                                               xvfb          selenium        0            20911
                                                                             2                1
                                  macOS 14.0   headless-new  selenium        0            10471
                                                                             2                1
                                               real          selenium        0            10456
firefox 119                       Ubuntu 22.04 headless      selenium        0            20912
                                               xvfb          selenium        0            20911
                                                                             2                1
                                  macOS 14.0   headless      selenium        0            10456
                                               real          selenium        0            10456
safari  17.0                      macOS 14.0   real          selenium        0            10753
                                                                             2               87

,id,outcome_type,outcome_value,test_name,test_status,test_message,test_stack,org_scheme,org_host,resp_scheme,...,version,headless_mode,os,automation_mode,add_info,outcome_str,clean_url,browser,org_origin,resp_origin
51278,51391,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,17.0,real,macOS 14.0,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/r...,safari macOS 14.0 17.0 selenium real,http://sub.headers.websec.saarland,https://headers.websec.saarland
51282,51395,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,17.0,real,macOS 14.0,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/r...,safari macOS 14.0 17.0 selenium real,http://sub.headers.websec.saarland,https://sub.sub.headers.websec.saarland
51444,51399,<class 'NoneType'>,None,referrer_iframe,2,Test timed out,None,http,sub.headers.websec.saarland,https,...,17.0,real,macOS 14.0,selenium,,None,http://sub.headers.websec.saarland/_hp/tests/r...,safari macOS 14.0 17.0 selenium real,http://sub.headers.websec.saarland,https://headers.webappsec.eu


In [54]:
# Create a json with all tests to redo (either timed out or completely missing)
# Idea: group by test and count number of unique browser_ids
# Get the maximum count
# For all tests that do not have the maximum count, compute which ones are missing
# Save them (unique page loads) to a file
# Caveat1: easy way will usually also result in the duplication of other tests (e.g., 5/10 tests on a page load timed out -> 10 tests will be repeated)
# However this should not matter too much as test results should be stable (see stability analysis below)
# Question: how to increase the TIMEOUT in these reruns to make the chances of additional time outs as low as possible?

all_browsers = set(df["browser_id"].unique())

def get_missing(browser_list):
    return all_browsers - set(browser_list)
    
browser_count = df.loc[df["test_status"] == 0].groupby(["test_name", "relation_info", "org_scheme", "org_host", "resp_scheme", "resp_host", "response_id", "clean_url"])["browser_id"].unique()
max_c = browser_count.apply(len).max()
missing = browser_count.loc[browser_count.apply(len) != max_c].apply(get_missing)
to_repeat = {}
for (test_name, relation_info, org_scheme, org_host, resp_scheme, resp_host, response_id, clean_url), row in missing.to_frame().iterrows():
    browser_ids = row.iloc[0]
    for browser_id in browser_ids:
        browser_id = str(browser_id)
        try:
            d = to_repeat[browser_id]
        except KeyError:
            d = set()
        d.add(re.sub("browser_id=(\d+)", f"browser_id={browser_id}", clean_url))
        to_repeat[browser_id] = d
with open("repeat.json", "w") as f:
    json.dump(to_repeat, f, default=list)

In [13]:
# Time taken!
# NOTE: this is now broken as we ran all tests on linux twice for a stability analysis
df.groupby(["name", "version", "os", "headless_mode", "automation_mode", "org_scheme"])["created_at"].agg(["min", "max"]).apply(lambda x: x["max"] - x["min"], axis=1).to_frame()

0
name    version                   os           headless_mode automation_mode org_scheme                       
brave   1.59.120 (118.0.5993.88)  Ubuntu 22.04 headless-new  selenium        http       0 days 06:03:09.397113
                                                                             https      0 days 06:00:44.089955
                                               xvfb          selenium        http       0 days 06:03:26.075669
                                                                             https      0 days 06:01:08.120806
        1.60.114 (119.0.6045.124) Ubuntu 22.04 headless-new  selenium        http       0 days 06:02:59.217855
                                                                             https      0 days 06:00:33.665142
                                               xvfb          selenium        http       0 days 06:03:30.937936
                                                                             https      0 days 06:01:01.203507
        1.60.118 (119.0.6045.163) macOS 14.0   headless-new  selenium        http       0 days 00:16:00.851072
                                                                             https      0 days 00:13:17.676180
                                               real          selenium        http       0 days 00:16:47.333179
                                                                             https      0 days 00:13:51.071158
chrome  119                       Ubuntu 22.04 headless-new  selenium        http       0 days 06:03:10.795015
                                                                             https      0 days 06:00:51.594540
                                               xvfb          selenium        http       0 days 06:02:34.495703
                                                                             https      0 days 06:00:15.101474
                                  macOS 14.0   headless-new  selenium        http       0 days 00:10:32.869022
                                                                             https      0 days 00:13:15.386574
                                               real          selenium        http       0 days 00:11:05.393402
                                                                             https      0 days 00:13:43.572152
edge    119                       Ubuntu 22.04 headless-new  selenium        http       0 days 05:58:12.727191
                                                                             https      0 days 06:01:12.427992
                                               xvfb          selenium        http       0 days 05:57:54.711679
                                                                             https      0 days 06:01:04.616337
                                  macOS 14.0   headless-new  selenium        http       1 days 00:13:42.604564
                                                                             https      0 days 00:13:29.270315
                                               real          selenium        http       0 days 00:11:14.907553
                                                                             https      0 days 00:13:58.131905
firefox 119                       Ubuntu 22.04 headless      selenium        http       0 days 05:59:39.733015
                                                                             https      0 days 06:02:24.061308
                                               xvfb          selenium        http       0 days 05:59:03.385511
                                                                             https      0 days 06:02:13.227681
                                  macOS 14.0   headless      selenium        http       0 days 00:10:44.408936
                                                                             https      0 days 00:14:12.761382
                                               real          selenium        http       0 days 00:10:48.618109
  

## Response analysis

- We have 10456 unique tests: (test_name x relation_info) x org_scheme x (resp_scheme x resp_host) x response_id)
- They should all have 19 (num browsers in various settings) outcomes

In [14]:
# Unique tests
tests = {
    "Fetch": [4, 2, 8, 6], 
    "Framing": [9, 2, 8, 35], # 7+11+17
    "PP": [4, 2, 8, 24],
    "OAC": [3, 2, 8, 14],
    "TAO": [1, 2, 8, 13],
    "RP": [2, 2, 8, 16],
    "CSP-SCRIPT": [2, 2, 8, 17],
    "COEP": [2, 2, 8, 11],
    "CORP": [4, 2, 8, 11],
    "CSP-IMG": [1, 2, 8, 17],
    "HSTS": [2, 1, 2, 14],
    "COOP": [1, 2, 8, 11],
}
tests = pd.DataFrame.from_dict(tests, columns=["tests", "org_origins", "resp_origins", "responses"], orient="index")
tests["All tests"] = tests.prod(axis=1)
tests.loc["Sum"] = tests.sum(axis=0)
tests

[autoreload of tree failed: Traceback (most recent call last):
  File "/home/ubuntu/.cache/pypoetry/virtualenvs/hp-OMYWa0k7-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 276, in check
    superreload(m, reload, self.old_objects)
  File "/home/ubuntu/.cache/pypoetry/virtualenvs/hp-OMYWa0k7-py3.11/lib/python3.11/site-packages/IPython/extensions/autoreload.py", line 475, in superreload
    module = reload(module)
             ^^^^^^^^^^^^^^
  File "/home/ubuntu/.pyenv/versions/3.11.5/lib/python3.11/importlib/__init__.py", line 169, in reload
    _bootstrap._exec(spec, module)
  File "<frozen importlib._bootstrap>", line 621, in _exec
  File "<frozen importlib._bootstrap_external>", line 940, in exec_module
  File "<frozen importlib._bootstrap>", line 241, in _call_with_frames_removed
  File "/home/ubuntu/wpt/_hp/tools/analysis/tree.py", line 9, in <module>
    from chefboost import Chefboost as chef
ModuleNotFoundError: No module named 'chefboost'
]


,tests,org_origins,resp_origins,responses,All tests
Fetch,4,2,8,6,384
Framing,9,2,8,35,5040
PP,4,2,8,24,1536
OAC,3,2,8,14,672
TAO,1,2,8,13,208
RP,2,2,8,16,512
CSP-SCRIPT,2,2,8,17,544
COEP,2,2,8,11,352
CORP,4,2,8,11,704
CSP-IMG,1,2,8,17,272


In [15]:
# Only consider the tests that did not timeout! (we already reran them above)
df = df.loc[df["test_status"] == 0]

In [16]:
# SELECT count(*) as c2, count(DISTINCT browser_id), count(DISTINCT outcome_value) as c1, test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id 
# FROM "Result" GROUP by test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id order by c1 DESC, c2 DESC

res = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])[["browser_id", "outcome_str"]].agg(["count", "nunique"])
display(res)
display(res.value_counts().to_frame())

browser_id  \
                                                                                                                                    count   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id              
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        70                  29   
                                                                                                                   71                  29   
                                                                                                                   72                  29   
                                                                                                                   73                  29   
                                                                                                                   74                  29   
...                                                                                                                                   ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     138                 29   
                                                                                                                   139                 29   
                                                                                                                   140                 29   
                                                                                                                   141                 29   
                                                                                                                   142                 29   

                                                                                                                                        \
                                                                                                                               nunique   
test_name             org_scheme org_host                    resp_scheme resp_host                   relation_info response_id           
accesswindow_direct   http       sub.headers.websec.saarland http        headers.webappsec.eu        direct        70               19   
                                                                                                                   71               19   
                                                                                                                   72               19   
                                                                                                                   73               19   
                                                                                                                   74               19   
...                                                                                                                                ...   
upgradeHSTS_subdomain http       sub.headers.websec.saarland https       sub.headers.websec.saarland subdomain     138              19   
                                                                                                                   139              19   
                                                                                                                   140              19   
                                                                                                                   141              19   
                                                                                                                   142              19   

                                                                                                                               outcome_str  \
                                                                             

count
(browser_id, count) (browser_id, nunique) (outcome_str, count) (outcome_str, nunique)       
29                  19                    29                   1                        8207
                                                               2                        1788
                                                               3                         289
30                  19                    30                   1                          59
31                  19                    31                   1                          56
32                  19                    32                   1                          23
31                  19                    31                   2                          16
32                  19                    32                   2                           7
                                                               3                           3
29                  19                    29                   4                           3
31                  19                    31                   3                           2
28                  19                    28                   1                           1
31                  19                    31                   4                           1
32                  19                    32                   4                           1

## Stability analysis
- Does the same browser/env always result in the same outcomes?
- We ran all tests on linux twice, does any of these tests have more than one result?
    - A total of 20/10456 tests have different outcomes
    - ~~2xreferrer_iframe (both timeout of the test "None" vs "message timeout" result); can be fixed by removing timeouts first~~
    - 18x subresourceloadingCOEP_img, all differences: 'swag-same-origin.jpg': 'error' | 'load'
        - trying to manually reproduce: e.g., http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=57&last_id=57&scheme=http
        - devtools open: with cache disabeled: 15x error, without cache disabled: 13x or 16x error?
        - devtools closed: 13x or 16x error (tried to fix with Cache-Control: no-cache, no-store but does not seem to have an effect)
        - chrome always 14x error
    - 13x firefox xvfb, 6x firefox headless, 1x edge

In [17]:
stab = df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id", "browser_id"])[["outcome_str"]].agg(["count", "nunique"])
# Tests with more than one outcome
diff_outcomes = stab.loc[stab[("outcome_str", "nunique")] != 1]
print(f"{len(diff_outcomes)} tests have different outcomes!")
display(diff_outcomes.reset_index()["browser_id"].apply(id_to_browser).value_counts().to_frame())
display(diff_outcomes)

18 tests have different outcomes!


,count
browser_id,
firefox Ubuntu 22.04 119 selenium xvfb,12
firefox Ubuntu 22.04 119 selenium headless,6


outcome_str  \
                                                                                                                                                         count   
test_name                  org_scheme org_host                    resp_scheme resp_host                       relation_info response_id browser_id               
subresourceloadingCOEP_img http       sub.headers.websec.saarland http        headers.websec.saarland         direct        57          72                   2   
                                                                                                                            58          72                   2   
                                                                                                                            59          14                   2   
                                                                                                                            60          72                   2   
                                                                                                                            62          14                   2   
                                                                              sub.headers.websec.saarland     direct        57          72                   2   
                                                                                                                            58          72                   2   
                                                                                                                            59          14                   2   
                                                                                                                            60          72                   2   
                                                                                                                            62          14                   2   
                                                                              sub.sub.headers.websec.saarland direct        57          72                   2   
                                                                                                                            58          72                   2   
                                                                                                                            59          14                   2   
                                                                                                                            60          72                   2   
                                                                                                                            62          14                   2   
                           https      sub.headers.websec.saarland https       headers.websec.saarland         direct        60          72                   2   
                                                                              sub.headers.websec.saarland     direct        60          72                   2   
                                                                              sub.sub.headers.websec.saarland direct        60          72                   2   

                                                                                                                                                            
                                                                                                                                                   nunique  
test_name                  org_scheme org_host                    resp_scheme resp_host                       relation_info response_id browser_id          
subresourceloadingCOEP_img http       sub.headers.websec.saarland http        headers.websec.saarland         direct        57          72               2  
                                                                                                                    

In [18]:
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id, browser_id in stab.loc[stab[("outcome_str", "nunique")] != 1].index:
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id) & (df["browser_id"] == browser_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        

subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
134639,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=57&last_id=57&scheme=http
265842,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'unsafe-none']]",subresourceloadingCOEP_img,http,sub.headers.websec.saarland,http,direct,headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=57&last_id=57&scheme=http


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


...

,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
145385,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'cross-origin']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=60&last_id=60&scheme=https
271131,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'cross-origin']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=60&last_id=60&scheme=https


subresourceloadingCOEP_img


,,count
outcome_str,name,
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}",firefox,1
"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}",firefox,1


,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
145509,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'load'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'cross-origin']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=60&last_id=60&scheme=https
271139,72,xvfb,firefox,"{'image-events': {'swag.jpg': 'load', 'swag-same-site.jpg': 'load', 'swag-cross-site.jpg': 'load', 'swag-same-origin.jpg': 'error'}, 'window.crossOriginIsolated': False}","[['Cross-Origin-Embedder-Policy', 'cross-origin']]",subresourceloadingCOEP_img,https,sub.headers.websec.saarland,https,direct,sub.sub.headers.websec.saarland,https://sub.headers.websec.saarland/_hp/tests/subresource-loading-coep.sub.html?resp_type=basic&browser_id=2&label=COEP&first_id=60&last_id=60&scheme=https


## Differences:
- COOP (accesswindow_direct):
    - [ ] Edge + Firefox: redirect first
    - [ ] Brave + Chrome: COOP first?
-  Framing:
    -  [ ] framing_iframe_sandbox: CSP-FA *; Firefox allows, all other browsers deny?
- PP not supported in Firefox
- OAC not supported in Firefox
    - [ ] Also random/caching behavior?
- PerfAPI:
    - [ ] Chrome/Firefox have different rules when to create an entry at all?
- COEP (CORP):
    - [ ] Firefox incorrect behavior for same-origin (applies same-site?)
- CORP:
    - [ ] same-origin: Edge does not check the scheme?
    - [x] Firefox applies CORP on object tags (even though it should not!)
    - [ ] Some other CORP and/or COEP strange stuff (Firefox)
- Referrer Policy: different default in Brave (unsafe-url is never allowed)
- Subresource-loading (CSP): differences due to HTTPS Upgrades
- Script Execution (CSP):
    - [ ] Safari sandbox bug?! (maybe related https://bugs.webkit.org/show_bug.cgi?id=223848 unsure though)
- Upgrade (HSTS):
    - [ ] Safari bug?

In [19]:
def get_uniques(df):
    browsers = f"{df['name'].unique().tolist()}"
    try:
        outcome = unique_outcomes[df.name[0]]
    except KeyError:
        outcome = {}
        #outcome = []
    outcome[browsers] = df.name[1]
    #outcome.append(f"{df.name[1]} ({browsers})")
    unique_outcomes[df.name[0]] = outcome

for grouping, group in df.groupby(["test_name", "relation_info"]):
    
    unique_outcomes = {}
    group.groupby(["name"])["outcome_str"].value_counts(normalize=True).reset_index().groupby(["outcome_str", "proportion"], group_keys=True).apply(get_uniques)
    d = pd.DataFrame.from_dict(unique_outcomes, orient="index")
    if d.shape[1] != 1:
        print(grouping)
        # display(unique_outcomes)
        display(group.groupby(["name"])["outcome_str"].value_counts(normalize=True).to_frame().unstack(0))
        for value, dic in unique_outcomes.items():
            values = [f"{v} ({k})" for k,v in dic.items()]
            # print(f"{value}: {values}")
        display(d)
        print()


('accesswindow_direct', 'direct')


proportion            \
name                                                    brave    chrome   
outcome_str                                                               
{'window.open.opener': 'null'}                       0.340909  0.287879   
{'window.open.opener': 'object "[object Window]"'}   0.659091  0.712121   

                                                                        \
name                                                    edge   firefox   
outcome_str                                                              
{'window.open.opener': 'null'}                      0.181818  0.181818   
{'window.open.opener': 'object "[object Window]"'}  0.818182  0.818182   

                                                              
name                                                  safari  
outcome_str                                                   
{'window.open.opener': 'null'}                      0.181818  
{'window.open.opener': 'object "[object Window]"'}  0.818182

,"['edge', 'firefox', 'safari']",['chrome'],['brave']
{'window.open.opener': 'null'},0.181818,0.287879,0.340909
"{'window.open.opener': 'object ""[object Window]""'}",0.818182,0.712121,0.659091



('framing_embed', 'direct')


proportion                                     
name                 brave    chrome      edge   firefox safari
outcome_str                                                    
message send      0.332143  0.332143  0.332143  0.332143    NaN
message timeout   0.667857  0.667857  0.667857  0.667857    1.0

...

proportion                              
name                         brave chrome  edge   firefox safari
outcome_str                                                     
error                         0.25   0.25  0.25  0.431818    NaN
load                          0.75   0.75  0.75  0.568182    NaN
timeout: no event fired        NaN    NaN   NaN       NaN    1.0

,"['brave', 'chrome', 'edge']",['firefox'],['safari']
error,0.25,0.431818,NaN
load,0.75,0.568182,NaN
timeout: no event fired,NaN,NaN,1.0



('upgradeHSTS_direct', 'direct')


proportion                                \
name                                brave    chrome      edge   firefox   
outcome_str                                                               
{'response.redirected': False}   0.821429  0.821429  0.821429  0.821429   
{'response.redirected': True}    0.178571  0.178571  0.178571  0.178571   

                                          
name                              safari  
outcome_str                               
{'response.redirected': False}  0.785714  
{'response.redirected': True}   0.214286

,['safari'],"['brave', 'chrome', 'edge', 'firefox']"
{'response.redirected': False},0.785714,0.821429
{'response.redirected': True},0.214286,0.178571


In [20]:
test_name_seen = set()
for test_name, org_scheme, org_host, resp_scheme, resp_host, relation_info, response_id in res.loc[res[("outcome_str", "nunique")] != 1].index:
    # Only look at each feature group once
    if test_name in test_name_seen:
        continue
    test_name_seen.add(test_name)
    print(test_name)
    rows = df.loc[(df["org_scheme"] == org_scheme) & (df["test_name"] == test_name) & (df["resp_scheme"] == resp_scheme) & (df["resp_host"] == resp_host) & (df["relation_info"] == relation_info) & (df["response_id"] == response_id)]
    with pd.option_context("display.max_colwidth", 200):
        #display(rows[["outcome_str", "name", "headless_mode", "os", "version"]].value_counts().to_frame())
        display(rows[["outcome_str", "name"]].value_counts().to_frame())
        disp = rows.drop_duplicates(subset="outcome_str")[["browser_id", "headless_mode", "name", "outcome_str", "raw_header", "test_name", "org_scheme", "org_host", "resp_scheme", "relation_info", "resp_host", "full_url"]]
        disp = disp.style.format({'full_url': make_clickable})
        display(disp)
        
    

accesswindow_direct


count
outcome_str                                        name          
{'window.open.opener': 'null'}                     brave       10
{'window.open.opener': 'object "[object Window]"'} edge         6
                                                   firefox      6
{'window.open.opener': 'null'}                     chrome       4
{'window.open.opener': 'object "[object Window]"'} chrome       2
                                                   safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
8245,13,headless-new,chrome,{'window.open.opener': 'null'},"[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=2&label=COOP&first_id=71&last_id=71&scheme=http
13276,5,real,chrome,"{'window.open.opener': 'object ""[object Window]""'}","[['Cross-Origin-Opener-Policy', 'same-origin-allow-popups']]",accesswindow_direct,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/window-references-coop.sub.html?resp_type=basic&browser_id=2&label=COOP&first_id=71&last_id=71&scheme=http


framing_embed


count
outcome_str     name          
message send    brave       10
                chrome       6
                edge         6
                firefox      6
message timeout safari       1

...

count
outcome_str             name          
load                    brave       10
                        chrome       6
                        edge         6
                        firefox      6
timeout: no event fired safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
6617,13,headless-new,chrome,load,"[['Cross-Origin-Resource-Policy', 'unsafe-none']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=2&label=CORP&first_id=44&last_id=44&scheme=http
54381,7,real,safari,timeout: no event fired,"[['Cross-Origin-Resource-Policy', 'unsafe-none']]",subresourceloadingCORP_object,http,sub.headers.websec.saarland,http,direct,headers.webappsec.eu,http://sub.headers.websec.saarland/_hp/tests/subresource-loading-corp.sub.html?resp_type=basic&browser_id=2&label=CORP&first_id=44&last_id=44&scheme=http


upgradeHSTS_direct


count
outcome_str                    name          
{'response.redirected': False} brave       10
                               chrome       6
                               edge         6
                               firefox      6
{'response.redirected': True}  safari       1

,browser_id,headless_mode,name,outcome_str,raw_header,test_name,org_scheme,org_host,resp_scheme,relation_info,resp_host,full_url
9260,13,headless-new,chrome,{'response.redirected': False},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_direct,http,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=2&label=HSTS&first_id=139&last_id=139&scheme=http
55979,7,real,safari,{'response.redirected': True},"[['Strict-Transport-Security', 'max-age=20, max-age=20; includeSubDomains, includeSubDomains, , max-age=20; includeSubDomains; preload, max-age=0, max-age=-5']]",upgradeHSTS_direct,http,sub.headers.websec.saarland,https,direct,sub.headers.websec.saarland,http://sub.headers.websec.saarland/_hp/tests/upgrade-hsts.sub.html?resp_type=basic&browser_id=2&label=HSTS&first_id=139&last_id=139&scheme=http


## Clustering and co.
- Maybe try decision trees again?
- One tree for each feature group?/label?/test_name?
    - outcome = outcome_str (mostly binary, sometimes more options)
    - variables:
        - browser (includes os, automation mode, headless mode, version)
        - test_id = test_name + relation_info (or only relation_info if one tree per test_name is created)
        - org_orgin = org_scheme + org_host
        - resp_origin = resp_scheme + resp_host
        - response_id or raw_headers + status_code
        - ...
     
- H2O setup:
    - `sudo apt install openjdk-17-jre-headless graphviz`
    - `wget https://h2o-release.s3.amazonaws.com/h2o/rel-3.44.0/2/h2o-3.44.0.2.zip`
    - `unzip h2o-3.44.0.2.zip`
    - `cd h2o-3.44.0.2` and `java -jar h2o.jar`

In [21]:
df[["test_name", "label", "relation_info"]].value_counts()

test_name                      label       relation_info 
fullscreen_iframe              PP          child_allow       11136
                                           child             11136
                                           direct            11136
                                           child_sandbox     11136
framing_iframe                 CSP-FA      nested             7888
                                           sandbox            7888
framing_object                 CSP-FA      sandbox            7888
                                           direct             7888
                                           nested             7888
framing_iframe                 CSP-FA      direct             7888
framing_embed                  CSP-FA      sandbox            7888
script_execution_iframe        CSP-SCRIPT  direct             7888
                                           sandbox            7888
imgloading_iframe              CSP-IMG     direct             7888
fram

In [53]:
h2o.init(nthreads=50, max_mem_size="100G", log_level="WARN")
h2o.no_progress()


# Only keep one row for each test that had non-deterministic results (this should only be Firefox COEP same-origin 18x)
print(len(df))
tree_df = df.drop_duplicates(subset=["test_name", "relation_info", "browser", "org_origin", "resp_origin", "response_id"])
print(len(tree_df))

# Idea limit on responses that have more than one outcome!
# This will remove all the branches that are the same in all browsers (and make trees large and confusing)
condition = tree_df.groupby(["test_name", "org_scheme", "org_host", "resp_scheme", "resp_host", "relation_info", "response_id"])["outcome_str"].transform("nunique") != 1
tree_df = tree_df.loc[condition]
print(len(tree_df))

for group_name, group in tree_df.groupby(["test_name", "relation_info"]):
    print(group_name)
    # pred_cols = ["browser", "relation_info", "org_origin", "resp_origin", "status_code", "raw_header"]
    pred_cols = ["browser", "org_origin", "resp_origin", "status_code", "raw_header"]

    group = group[["outcome_str", *pred_cols]]
    # Convert all columns to str, then to category
    tree = make_tree(group, pred_cols, group_name)
    # print(tree)

# Try the other tree library that allows non-binary splits?



Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O_cluster_uptime:,5 hours 18 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_ubuntu_pmk8uo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.1 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


303534
198664
40052
('accesswindow_direct', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,5 hours 18 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_ubuntu_pmk8uo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.1 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('accesswindow_direct', 'direct')
('framing_embed', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


...

H2O_cluster_uptime:,5 hours 18 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_ubuntu_pmk8uo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.1 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCOEP_img', 'sandbox')
('subresourceloadingCORP_img', 'direct')
Connecting to H2O server at http://localhost:54321 ... successful.


H2O_cluster_uptime:,5 hours 18 mins
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.44.0.2
H2O_cluster_version_age:,15 days
H2O_cluster_name:,H2O_from_python_ubuntu_pmk8uo
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,99.1 Gb
H2O_cluster_total_cores:,160
H2O_cluster_allowed_cores:,50
H2O_cluster_status:,"locked, healthy"


Create tree: ('subresourceloadingCORP_img', 'direct')


OSError: Job with key $03017f00000132d4ffffffff$_bbb230b0ba52ea277ebfa7e9902d4cc6 failed with an exception: java.lang.AssertionError
stacktrace: 
java.lang.AssertionError
	at hex.FoldAssignment.fromInternalFold(FoldAssignment.java:47)
	at hex.ModelBuilder.cv_AssignFold(ModelBuilder.java:712)
	at hex.ModelBuilder.computeCrossValidation(ModelBuilder.java:618)
	at hex.ModelBuilder$1.compute2(ModelBuilder.java:392)
	at water.H2O$H2OCountedCompleter.compute(H2O.java:1689)
	at jsr166y.CountedCompleter.exec(CountedCompleter.java:468)
	at jsr166y.ForkJoinTask.doExec(ForkJoinTask.java:263)
	at jsr166y.ForkJoinPool$WorkQueue.runTask(ForkJoinPool.java:976)
	at jsr166y.ForkJoinPool.runWorker(ForkJoinPool.java:1479)
	at jsr166y.ForkJoinWorkerThread.run(ForkJoinWorkerThread.java:104)


# Crash analysis

### Exceptions thrown
- Fixed:
    - Only Firefox; both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Response_ids: 185, 186, 341, 342; these are the ones that have status code 302!
        - Reason seems to be that Firefox switches to another tab in these cases and thus the wait for crashes. Solution: forcefully switch back to the original tab before waiting!
    - Only Brave: both headless + Xvfb
        - OAC + RP
        - Both on HTTP and HTTPS
        - Reason seems to be the tab switching, if deactivated no issue occurs

In [ ]:
with open("/home/ubuntu/wpt/_hp/tools/crawler/desktop-selenium-2023-11-20 16:09:24.649008.log", "r") as f:
    crashes = []
    for l in f.readlines():
        if "649008: http" in l:
            match = re.match(".*(http|https)://.*browser_id=(\d+)&label=(.*)&first_id=(\d+).*", l)
            crashes.append([match[1], match[2], match[3], match[4]])

In [ ]:
crashes = pd.DataFrame(crashes, columns=["scheme", "browser_id", "label", "response_id"]).drop_duplicates()
crashes